<a href="https://colab.research.google.com/github/awasekhirni/diseasemodels/blob/main/basemap_postalcodes_canada.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
#Copyright 2020 Awase Khirni Syed awase.syed@uwaterloo.com 
#awase008@gmail.com
#MULTISCALE DYNAMIC HUMAN MOBILITY FLOW PATTERNS FOR SPATIO TEMPORAL EPIDEMIOLOGY MODELING

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
!pip install geopandas
import geopandas as gpd
#!pip install osmnx
#import osmnx as ox
!pip install descartes 
from descartes import PolygonPatch
!pip install shapely
from shapely.geometry import Point, LineString, Polygon,MultiPolygon 

#retrieve tile maps from the internet 
import contextily as ctx 
#import library pickle # it is similar to json 
import pickle 


#loading the shapefiles using geopandas 
postal_canada = gpd.read_file("https://drive.google.com/file/d/1EuFGsL8PXTma2mCNLT7eenOnwTB8Ntgs/view?usp=sharing")

#printing the head of the file 
postal_canada.head()
#quick plot snapshot 
postal_canada.plot() 
#fetching the coordinate system information 
#gps => wgs64 is 4326 datum wgs84 
postal_canada.crs 
# locading the canada postal zone from open streetmap using osnmx api 
# fetching polygon by default open streetmap gives centroid point of the location 
osnmx_canada=ox.gdf_from_place('Canada',which_result=2)
print(osnmx_canada)
osnmx_canada=ox.project_gdf(osnmx_canada)
#plotting the data 
fig, ax_canada =ox.plot_shape(osnmx_canada)
#utm wgs84
print(osnmx_canada.crs)

#saving this for future use 
osnmx_canada.to_file('stagingarea/zipcode_canada.shp')
geometry=osnmx_canada['geometry'].iloc[0]
#identify the geometry info 
print(type(geometry))
#slice the geometry by 1000 meters resolution 
geometry_cut=ox.quadrat_cut_geometry(geometry,quadrat_width=1000)
print(geometry_cut)
#using list comprehension to generate a list of polygons 
can_polylist=[p for p in geometry_cut]
#drawing the canada postal polygons 
west,south,east,north = osnmx_canada.unary_union.bounds
fig,ax=plt.subplots(figsize=(50,50))
#enabling polygons 
for polygon, n in zip(geometry_cut, np.arange(len(can_polylist))):
  #representative point 
  print(polygon.representative_point().coords[:])
  p=polygon.representative_point().coords[:][0]
  #adding annotation layer using descartes 
  patch=PolygonPatch(polygon,fc="#ffffff",ec="#000000",alpha=0.5,zorder=2)
  ax.add_patch(patch)
  #annotate the labels to the centroid of the polygon 
  plt.annotate(s=n,xy=p, horizontalalignment='center',size=15)

#alignment settings 
ax.set_xlim(west,east)
ax.set_ylim(south,north)
ax.axis('off')
plt.show()

ModuleNotFoundError: ignored